# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff1d41a1df0>
├── 'a' --> tensor([[ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400]])
└── 'x' --> <FastTreeValue 0x7ff1d41a1f10>
    └── 'c' --> tensor([[-0.3935,  0.1973, -1.9234,  0.9019],
                        [-0.1539,  0.1133,  2.0211, -1.1916],
                        [-0.2434, -0.7556,  1.7643, -2.6533]])

In [4]:
t.a

tensor([[ 1.5005, -0.0279,  1.2275],
        [ 0.1263, -0.7852, -0.7400]])

In [5]:
%timeit t.a

58.7 ns ± 0.0611 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff1d41a1df0>
├── 'a' --> tensor([[-0.4757,  0.0644, -1.1868],
│                   [ 0.0038, -0.2724, -1.3462]])
└── 'x' --> <FastTreeValue 0x7ff1d41a1f10>
    └── 'c' --> tensor([[-0.3935,  0.1973, -1.9234,  0.9019],
                        [-0.1539,  0.1133,  2.0211, -1.1916],
                        [-0.2434, -0.7556,  1.7643, -2.6533]])

In [7]:
%timeit t.a = new_value

61.3 ns ± 0.0222 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5005, -0.0279,  1.2275],
               [ 0.1263, -0.7852, -0.7400]]),
    x: Batch(
           c: tensor([[-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5005, -0.0279,  1.2275],
        [ 0.1263, -0.7852, -0.7400]])

In [11]:
%timeit b.a

56.7 ns ± 0.0956 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2656,  1.5160,  1.1012],
               [-2.3341, -2.5560, -0.7017]]),
    x: Batch(
           c: tensor([[-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.0953 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

962 ns ± 8.43 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 652 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff11d4ac820>
├── 'a' --> tensor([[[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]],
│           
│                   [[ 1.5005, -0.0279,  1.2275],
│                    [ 0.1263, -0.7852, -0.7400]]])
└── 'x' --> <FastTreeValue 0x7ff11d4acc10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 49.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff11d4ac8b0>
├── 'a' --> tensor([[ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400],
│                   [ 1.5005, -0.0279,  1.2275],
│                   [ 0.1263, -0.7852, -0.7400]])
└── 'x' --> <FastTreeValue 0x7ff11d4ac730>
    └── 'c' --> tensor([[-0.3935,  0.1973, -1.9234,  0.9019],
                        [-0.1539,  0.1133,  2.0211, -1.1916],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 84 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.6 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.3935,  0.1973, -1.9234,  0.9019],
                       [-0.1539,  0.1133,  2.0211, -1.1916],
                       [-0.2434, -0.7556,  1.7643, -2.6533]],
              
                      [[-0.3935,  0.1973, -1.9234,  0.9019],
                       [-0.1539,  0.1133,  2.0211, -1.1916],
                       [-0.2434, -0.7556,  1.7643, -2.6533]],
              
                      [[-0.3935,  0.1973, -1.9234,  0.9019],
                       [-0.1539,  0.1133,  2.0211, -1.1916],
                       [-0.2434, -0.7556,  1.7643, -2.6533]],
              
                      [[-0.3935,  0.1973, -1.9234,  0.9019],
                       [-0.1539,  0.1133,  2.0211, -1.1916],
                       [-0.2434, -0.7556,  1.7643, -2.6533]],
              
                      [[-0.3935,  0.1973, -1.9234,  0.9019],
                       [-0.1539,  0.1133,  2.0211, -1.1916],
                       [-0.2434, -0.7556,  1.7643, -2.6533]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533],
                      [-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533],
                      [-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533],
                      [-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533],
                      [-0.3935,  0.1973, -1.9234,  0.9019],
                      [-0.1539,  0.1133,  2.0211, -1.1916],
                      [-0.2434, -0.7556,  1.7643, -2.6533],
                      [-0.3935,  0.1973, -1.9234,  0.9019],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
